# Imports

In [1]:
import warnings
import os
import sys
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# Realod modules
import importlib

In [27]:
import sys
sys.path.append(r'C:\Users\Lenovo\Documents\DSPython\projetscoring')

In [3]:
# Data Ingestion : import des data et autres fonctionnalité
from src.components.data_ingestion import *

In [4]:
# Data Transformation : Transformation des data
from src.components.data_transformation import DataTransformation

In [5]:
from src.components.model_trainer import *

In [6]:
from src.utilis import save_object, evaluate_model_

# Relaod modules

In [14]:
importlib.reload(sys.modules['src.components.data_ingestion'])
from src.components.data_ingestion import *
importlib.reload(sys.modules['src.components.data_transformation'])
from src.components.data_transformation import *
importlib.reload(sys.modules['src.components.model_trainer'])
from src.components.model_trainer import *

# Data base

In [7]:
# 01 - Data Base : 
# Le chemin pour accéder à la base des données se trouve dans les configurations : data_config.py
data_base = DataIngestion()

In [24]:
# Extraction nom de fichiers avec et sans extensions
liste_name, files_liste_name = data_base.get_files_names()

In [25]:
liste_name

['application_test',
 'application_train',
 'bureau',
 'bureau_balance',
 'credit_card_balance',
 'Final_df',
 'HomeCredit_columns_description',
 'installments_payments',
 'POS_CASH_balance',
 'previous_application',
 'sample_submission']

In [21]:
files_liste_name

['application_test.csv',
 'application_train.csv',
 'bureau.csv',
 'bureau_balance.csv',
 'credit_card_balance.csv',
 'Final_df.csv',
 'HomeCredit_columns_description.csv',
 'installments_payments.csv',
 'POS_CASH_balance.csv',
 'previous_application.csv',
 'sample_submission.csv']

# Import data

In [22]:
# df = pd.read_csv('C:/Users/Lenovo/Documents/DSPython/data_projet_7/Final_df.csv')

In [8]:
row_data = data_base.import_file(file_name='Final_df.csv', reduce_memory_usage = False)

Importation du dataset...
Importation du dataset réussie !


In [9]:
row_data.head()

,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,1,0,0,0,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
4,0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
rapport_df_row_data = RapportDataFrame(row_data, target_column="TARGET", ID_Columns=["SK_ID_CURR"])
# Rapport : row_datarow_data
print("Data Frame : row_data")
print(20 * "--")
rapport_df_row_data.rapport(nan_threshold = 20, return_column_to_keep=False, print_rapport = True)

Data Frame : row_data
----------------------------------------
Nombre de ligne : 307507 --- Nombre de colonnes : 798
----------------------------------------
Le Taux de remplissage total est égal à : 74.76 %
Nombre de colonnes ayant moins de 20% de valeurs manquantes : 517
Le Nombre de features vides est égal à : 0 Features
----------------------------------------
Les Features vides sont : []
Les Features Ayant plus  20%  de valeurs manquantes sont:
['REFUSED_RATE_DOWN_PAYMENT_MIN', 'REFUSED_RATE_DOWN_PAYMENT_MAX', 'REFUSED_AMT_DOWN_PAYMENT_MEAN', 'REFUSED_AMT_DOWN_PAYMENT_MAX', 'REFUSED_AMT_DOWN_PAYMENT_MIN', 'REFUSED_RATE_DOWN_PAYMENT_MEAN', 'REFUSED_APP_CREDIT_PERC_VAR', 'CC_AMT_PAYMENT_CURRENT_VAR', 'CC_CNT_DRAWINGS_ATM_CURRENT_VAR', 'CC_AMT_DRAWINGS_OTHER_CURRENT_VAR', 'CC_AMT_DRAWINGS_POS_CURRENT_VAR', 'CC_AMT_DRAWINGS_ATM_CURRENT_VAR', 'CC_CNT_DRAWINGS_POS_CURRENT_VAR', 'CC_CNT_DRAWINGS_OTHER_CURRENT_VAR', 'CC_AMT_PAYMENT_CURRENT_MEAN', 'CC_AMT_PAYMENT_CURRENT_MAX', 'CC_AMT_PAYM

# Data transformation

In [10]:
transformer_row_data= DataTransformation(row_data)

# Train Test Split

In [11]:
X_train, X_test, y_train, y_test = transformer_row_data.initiate_train_test_split(
    target="TARGET",
    test_size=0.30,
    stratification=True)

X_train : done 1/4 ---- Nombre de lignes :215254, Nombre de colonnes:795
y_train : done 2/4 ---- Nombre de lignes :215254
X_test : done 3/4 ---- Nombre de lignes :92253, Nombre de colonnes:795
y_test : done 4/4 ---- Nombre de lignes :92253
####################
Les données sont enregistrées dans le dossier artifacts


In [31]:
X_train.shape

(215254, 797)

# Preprocessing

In [10]:
# Si les data frames sont existants alors exécuter la cellule pour les importer
# transformer_row_data= DataTransformation()
# X_train,X_test, y_train, y_test = transformer_row_data.get_data_frames()

In [12]:
# Pour initier la transformation des données 
# X_tra, X_test, y_train, y_test retournés : sont transformés avec une pipeline définie dans le fichier data_transformation
# avec la méthode initiate_data_transformation
# Les étape de la pipeline sont : 

# 1- application RandomUnderSampler(random_state=0)
# 2- num_pipeline
# 3- categ_piepline
# 4- fillna des valeur .inf -.inf
# 5- fit du preprocessor sur les données X_train
# 6- transform des données X_train et X_test
# 7- Save du preprocessor dans le dossier artifacts
# 8- Return X_train, X_test, y_train, y_test preprocssed
# transformer_row_data= DataTransformation()
X_train, X_test,y_train,y_test= transformer_row_data.initiate_data_transformation(
    X_train, 
    X_test, 
    y_train,
    y_test,
    undersampling= True,
    return_train_test_array=False
)

TARGET distribution avant Undersampling
0    197877
1     17377
Name: TARGET, dtype: int64

TARGET distribution après Undersampling
0    17377
1    17377
Name: TARGET, dtype: int64

Les étapes de transformation :
ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaled', StandardScaler())]),
                                 ['CODE_GENDER', 'FLAG_OWN_CAR',
                                  'FLAG_OWN_REALTY', 'CNT_CHILDREN',
                                  'AMT_INCOME_TOTAL', 'AMT_CREDIT',
                                  'AMT_ANNUITY', 'AMT_GOODS_PRICE',
                                  'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH',
                                  'DAYS_EMPLOYED', 'DAYS_REGISTRATION',
                                  'DAYS_ID_PUBLISH', 'OWN_...
                                  'REG_REGION_NOT_WORK_REGION',
                                

In [13]:
X_train.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,0.783909,-0.692704,1.490284,0.773952,-0.067317,1.123063,1.794167,1.110862,-0.757876,0.436659,...,0.000000,-2.420554e-17,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,4.031296e-16
1,0.783909,-0.692704,-0.671013,-0.594760,-0.067317,-1.061894,-1.315027,-0.979866,-0.795281,1.664291,...,0.000000,-2.420554e-17,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,4.031296e-16
2,0.783909,1.443619,-0.671013,3.511375,-0.035002,-0.191526,0.975386,-0.195843,-1.126896,0.436891,...,-0.402033,-2.033759e-01,-0.156366,-0.272536,0.0,0.0,0.0,0.0,0.0,-9.883663e-01
3,0.783909,1.443619,-0.671013,3.511375,-0.053267,0.599177,-0.062442,0.457509,0.194676,0.885770,...,-0.402033,-2.033759e-01,-0.156366,-0.272536,0.0,0.0,0.0,0.0,0.0,-1.442250e+00
4,0.783909,1.443619,1.490284,-0.594760,0.016984,1.311405,0.584468,1.110862,4.002670,0.391192,...,0.000000,-2.420554e-17,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,4.031296e-16


In [14]:
X_test.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,-1.275659,1.443619,1.490284,-0.594760,0.087235,-0.711083,-0.296992,-0.718525,0.655608,1.515361,...,0.0,-2.420554e-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.031296e-16
1,-1.275659,-0.692704,-0.671013,0.773952,-0.137568,-1.312065,-1.547268,-1.306542,0.194676,0.606004,...,0.0,-2.420554e-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.031296e-16
2,0.783909,-0.692704,-0.671013,-0.594760,-0.067317,-0.883540,-1.067611,-0.783860,0.859127,1.203813,...,0.0,-2.420554e-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.031296e-16
3,0.783909,-0.692704,-0.671013,-0.594760,-0.053267,-1.327094,-1.702975,-1.319609,0.815308,-1.194383,...,0.0,-2.420554e-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.031296e-16
4,0.783909,-0.692704,-0.671013,-0.594760,-0.074342,-0.526831,-0.572779,-0.391849,-0.156175,-1.682467,...,0.0,-2.420554e-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.031296e-16


In [20]:
y_train.shape

(34754, 1)

In [21]:
y_test.shape

(92253, 1)

# Model trainer

In [7]:
# X_train = pd.read_csv('./artifacts/X_train.csv')
# X_test = pd.read_csv('./artifacts/X_test.csv')
# y_train = pd.read_csv('./artifacts/y_train.csv')
# y_test = pd.read_csv('./artifacts/y_test.csv')

In [16]:
modeltrainer = ModelTrainer()

**Logistic Regression**

In [17]:
model = LogisticRegression()

params = {
        'penalty': ['l1', 'l2'],
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'solver': ['liblinear', 'saga']
        }
        
modeltrainer.initiate_model_trainer(X_train,y_train,X_test,y_test, model, params, save_best_model=True)

LogisticRegression -- test_accuracy: 0.7035218366882378 -- train_accuracy: 0.7123496575933705


In [16]:
from lazypredict.Supervised import LazyClassifier

In [17]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

100%|██████████| 29/29 [1:26:26<00:00, 178.85s/it]  


In [18]:
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.69,0.69,0.69,0.76,2.57
LinearDiscriminantAnalysis,0.69,0.68,0.68,0.76,4.76
RidgeClassifierCV,0.69,0.68,0.68,0.76,2.70
RidgeClassifier,0.69,0.68,0.68,0.76,1.16
LogisticRegression,0.69,0.68,0.68,0.76,2.47
XGBClassifier,0.68,0.68,0.68,0.75,10.14
CalibratedClassifierCV,0.68,0.68,0.68,0.76,111.17
AdaBoostClassifier,0.69,0.68,0.68,0.76,16.31
LinearSVC,0.68,0.68,0.68,0.75,30.74


In [19]:
predictions

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.69,0.69,0.69,0.76,2.57
LinearDiscriminantAnalysis,0.69,0.68,0.68,0.76,4.76
RidgeClassifierCV,0.69,0.68,0.68,0.76,2.70
RidgeClassifier,0.69,0.68,0.68,0.76,1.16
LogisticRegression,0.69,0.68,0.68,0.76,2.47
XGBClassifier,0.68,0.68,0.68,0.75,10.14
CalibratedClassifierCV,0.68,0.68,0.68,0.76,111.17
AdaBoostClassifier,0.69,0.68,0.68,0.76,16.31
LinearSVC,0.68,0.68,0.68,0.75,30.74


In [ ]:
model = KNeighborsClassifier()

params = {
        'n_neighbors': [3, 5, 7],
        'weights': ['uniform', 'distance'],
        'algorithm': ['ball_tree', 'kd_tree', 'brute']
    }
        
modeltrainer.initiate_model_trainer(X_train,y_train,X_test,y_test, model, params, save_best_model=False)

In [ ]:
# Ancienne version 

# # Initiation du fit et trainig 
# models = {
#                 "Logistic Regression" : LogisticRegression(),
#                 # "Naive Bayes" : GaussianNB(),
#                 # "Kneighbors Classifier" : KNeighborsClassifier(3),
# #                 "model_4" : model_4(),
# #                 "model_5" : model_5(),
# #                 "model_6" : model_6()
#             }

# params = {
#         "Logistic Regression": {
#         'penalty': ['l1', 'l2'],
#         'C': [0.001, 0.01, 0.1, 1, 10, 100],
#         'solver': ['liblinear', 'saga']
#         }
#     #     "Kneighbors Classifier":{
#     #     'n_neighbors': [3, 5, 7],
#     #     'weights': ['uniform', 'distance'],
#     #     'algorithm': ['ball_tree', 'kd_tree', 'brute']
#     # }
# }

# # Initiation du fit et trainig 
# models = {
#                 # "Logistic Regression" : LogisticRegression(),
#                 # "Naive Bayes" : GaussianNB(),
#                 "Kneighbors Classifier" : KNeighborsClassifier(3),
# #                 "model_4" : model_4(),
# #                 "model_5" : model_5(),
# #                 "model_6" : model_6()
#             }

# params = {
#         # "Logistic Regression": {
#         # 'penalty': ['l1', 'l2'],
#         # 'C': [0.001, 0.01, 0.1, 1, 10, 100],
#         # 'solver': ['liblinear', 'saga']
#         # },
#         "Kneighbors Classifier":{
#         'n_neighbors': [3, 5, 7],
#         'weights': ['uniform', 'distance'],
#         'algorithm': ['ball_tree', 'kd_tree', 'brute']
#     }
# }



# modeltrainer.initiate_model_trainer(X_tr= X_train, y_tr= y_train, X_te= X_test, y_te= y_test, model=models, param=params)